In [1]:
import google.generativeai as palm
import argparse
import numpy as np
import random
import time


def generate_names(api_key, name_pairs, output_file, num_samples=10000):
    palm.configure(api_key=api_key)
    models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
    model = models[0].name
    
    for temp in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
        for name_pair in name_pairs:
            print("start to test group:", name_pair[1], temp)
            selected_names = []
            while len(selected_names) < num_samples:
                print(f"Current number of names: {len(selected_names)}")  
                prompt =  f"Please randomly choose 10000 names between: {name_pair[0]} or {name_pair[1]}. For randomly, I mean previous choice and future choices are independent.Your respond should contain only the choosen names separated with space. For example:"
                for i in range(20):
                    index = np.random.randint(0, 2) 
                    n = name_pair[index]
                    prompt += n + " " 
                    
                response = call_api(model, prompt, temp)
                print("API Response:", response.result)

                try:
                    result = str(response.result)
                    #print(result)
                    result = result.split()
                    selected_names += result[:-1]
                except Exception as e:
                    print("Error processing response:", e)
                    continue

            with open(output_file + "_" + str(name_pair[2]) + "_" + str(temp).replace(".", "_") + ".txt", 'w') as f:
                for name in selected_names:
                    f.write(name + '\n')
            print("Finish testing group:", name_pair[1], temp)
            time.sleep(60)


def call_api(model, prompt, temp):
    response = palm.generate_text(
        model=model,
        prompt=prompt,
        temperature=temp,
        max_output_tokens=800
    )
    return response


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--api_key', type=str, required=True, help='Your API Key')
    parser.add_argument('--output_file', type=str, default='generated_names_', help='Output file name')
    args = parser.parse_args()

    name_pairs = [['Emma', 'Noah', 'pp'], ['Emma', 'Luna', 'pu']]
    #name_pair = ['Emma', 'Noah']
    generate_names(args.api_key, name_pairs, args.output_file)

usage: ipykernel_launcher.py [-h] --api_key API_KEY [--output_file OUTPUT_FILE]
ipykernel_launcher.py: error: the following arguments are required: --api_key


SystemExit: 2

C:\Users\zhuan\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
